# This software will:
    * Retweet any tweet
    * Schedule a new tweet
    * Edit a scheduled tweet
    

In [1]:
import config

consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret

# config.username
# config.password

# Import Tweepy
import tweepy

import logging
import os
import pickle
import re

logger = logging.getLogger()

def create_api():


# Set Authentication Credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

# Create API object
    api = tweepy.API(auth, wait_on_rate_limit=True,
                        wait_on_rate_limit_notify=True)

# Create API Object
    api = tweepy.API(auth)

    try:
        api.verify_credentials()
        print("Authentication OK")
    except Exception as e:
        logger.err0r("Error creating API", exc_info=True)
        raise e
#         print("Error during authentication")
    logger.info("API created")
    return api

In [2]:
create_api()

Authentication OK


In [22]:


# url = str(input("Please enter the url of the tweet you would like retweeted."))
url = 'https://twitter.com/KrakoaWelcomes/status/1268958023277518851'
id = re.findall(r'\d{19}$', url)
id[0]

'1268958023277518851'

In [23]:
def retweet():
    
    # Set Authentication Credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

# Create API object
    api = tweepy.API(auth, wait_on_rate_limit=True,
                        wait_on_rate_limit_notify=True)
    api = tweepy.API(auth)
    url = str(input("Please enter the url of the tweet you would like retweeted."))
    id = re.findall(r'\d{19}$', url)
    api.retweet(id[0])
    print('Your tweet has been successfully retweeted!')

retweet()
    
#    schedule post time
#  edit post time (delete scheduled post and create new scheduled tweet)

Please enter the url of the tweet you would like retweeted.https://twitter.com/KrakoaWelcomes/status/1268958023277518851
Your tweet has been successfully retweeted!


In [5]:
print(profile_image_url)

NameError: name 'profile_image_url' is not defined

In [7]:
import tweepy
from tweepy import OAuthHandler
import json
import config

consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_secret = config.access_token_secret
 
@classmethod
def parse(cls, api, raw):
    status = cls.first_parse(api, raw)
    setattr(status, 'json', json.dumps(raw))
    return status
 
# Status() is the data model for a tweet
tweepy.models.Status.first_parse = tweepy.models.Status.parse
tweepy.models.Status.parse = parse
# User() is the data model for a user profil
tweepy.models.User.first_parse = tweepy.models.User.parse
tweepy.models.User.parse = parse
# You need to do it for all the models you need
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [8]:
tweets = api.profile_image_url_https(screen_name='schwarm',
                           count=200, include_rts=False,
                           exclude_replies=True)

RecursionError: maximum recursion depth exceeded